In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
from langsmith import Client

client = Client()

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vector_store = Chroma(
    embedding_function=OpenAIEmbeddings(model='text-embedding-3-large'),
    collection_name='income_tax_collections',
    persist_directory='./income_tax_collections'
)
retriever = vector_store.as_retriever(search_kwargs={'k': 3})

In [ ]:
from typing import Literal
from typing_extensions import TypedDict
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

class AgentState(TypedDict):
    query: str
    context: list
    answer: str

In [ ]:
def retrieve(state: AgentState) -> AgentState:
    """
    'retrieve' Node
    : 사용자의 질문에 기반하여, 벡터 스토어에서 관련 문서를 검색한다.

    Args:
        - state(AgentState): 사용자의 질문을 포함한 에이전트의 현재 state

    Returns:
        - AgentState: 검색된 문서가 추가된 state
    """
    
    query = state['query']
    context = retriever.invoke(query)
    
    return {'context': context}

In [ ]:
rag_prompt = client.pull_prompt("rlm/rag-prompt", include_model=True)

def generate(state: AgentState) -> AgentState:
    """
    'generate' Node
    : 사용자의 질문과 검색된 문서를 기반으로 응답을 생성한다.

    Args:
        - state(AgentState): 사용자의 질문과 검색된 문서를 포함한 에이전트의 현재 state

    Returns:
        - AgentState: 생성된 응답이 추가된 state
    """
    
    query = state['query']
    context = state['context']
    
    rag_chain = rag_prompt | llm
    ai_message = rag_chain.invoke({'question': query, 'context': context})
    
    return {'answer': ai_message}

In [ ]:
rewrite_prompt = PromptTemplate.from_template(
    """
    사용자의 질문을 보고, 웹 검색에 용이하도록 질문을 변경해주세요.
    
    질문: {query}
    """
)

def rewrite(state: AgentState) -> AgentState:
    """
    'rewrite' Node
    : 사용자의 질문을 웹 검색용으로 변경한다.

    Args:
        - state(AgentState): 사용자의 질문을 포함한 에이전트의 현재 state

    Returns:
        - AgentState: 변경된 질문을 포함하는 state
    """
    
    query = state['query']
    
    rewrite_chain = rewrite_prompt | llm | StrOutputParser()
    ai_message = rewrite_chain.invoke({'query': query})
    
    return {'query': ai_message}

In [ ]:
doc_relevance_prompt = client.pull_prompt("langchain-ai/rag-document-relevance", include_model=True)

def check_doc_relevance(state: AgentState) -> Literal['relevant', 'irrelevant']:
    """
    : 주어진 state를 기반으로 문서의 관련성을 판단한다.

    Args:
        - state(AgentState): 사용자의 질문과 문맥을 포함한 에이전트의 현재 state

    Returns:
        - Literal['relevant', 'irrelevant']: 문서가 관련성이 높으면 'relevant', 그렇지 않으면 'irrelevant' 반환
    """
    
    query = state['query']
    context = state['context']
    
    doc_relevance_chain = doc_relevance_prompt | llm
    ai_message = doc_relevance_chain.invoke({'question': query, 'documents': context})
    
    ## node를 직접 지정하는 방식 대신 실제 판단 결과를 리턴함으로써 해당 node의 재사용성을 높일 수 있다.
    return 'relevant' if ai_message['Score'] == 1 else 'irrelevant'

In [ ]:
from langchain_community.tools import TavilySearchResults

tavily_search_tool = TavilySearchResults(
    max_results=3,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

In [ ]:
def web_search(state: AgentState) -> AgentState:
    """
    'web_search' Node
    : 주어진 state를 기반으로 웹 검색을 수행한다.

    Args:
        - state(AgentState): 사용자의 질문을 포함한 에이전트의 현재 state

    Returns:
        - AgentState: 웹 검색 결과가 추가된 state
    """
    
    query = state['query']
    
    # 웹 검색 도구 활용
    results = tavily_search_tool.invoke(query)
    
    return {'context': results}

In [ ]:
from langgraph.graph import StateGraph, START, END

graph_builder = StateGraph(AgentState)

# nodes
graph_builder.add_node('retrieve', retrieve)
graph_builder.add_node('generate', generate)
## graph_builder.add_node('rewrite', rewrite)
graph_builder.add_node('web_search', web_search)

# edges
graph_builder.add_edge(START, 'retrieve')
graph_builder.add_conditional_edges(
    'retrieve',
    check_doc_relevance,
    {
        'relevant': 'generate',
        'irrelevant': 'web_search'
    }
)
## graph_builder.add_edge('rewrite', 'web_search')
graph_builder.add_edge('web_search', 'generate')
graph_builder.add_edge('generate', END)

In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
query1 = "연봉 5천만원인 거주자가 납부해야 하는 소득세는 얼마인가요?"
query2 = "군자역 맛집을 알려주세요."
initial_state = {'query': query2}

graph.invoke(initial_state)